<a href="https://colab.research.google.com/github/LunaJhoeel/analisis-sentimiento/blob/main/extractor_comentarios_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx

In [2]:
import pandas as pd
from docx import Document
import xml.etree.ElementTree as ET

In [3]:
# ruta al archivo DOCX
ruta_docx = '/content/doc_prueba.docx'
print(f"Archivo {ruta_docx} DOCX cargado con éxito!")

Archivo /content/doc_prueba.docx DOCX cargado con éxito!


In [4]:
def extractor_parrafos(ruta):
    """Función que extrae los párrafos comentados de un docx
    Parámetro:
    ruta_docx: ruta del archivo

    Devuelve:
    - lista: lista de diccionarios, donde cada diccionario representa un párrafo con comentarios.
      Cada diccionario tiene las siguientes claves:
      - 'texto_completo': El texto completo del párrafo.
      - 'texto_comentado': La parte del párrafo que ha sido comentada.
    """
    doc = Document(ruta)
    xml_content = doc.part.blob
    xml_str = xml_content.decode("utf-8")
    raiz = ET.fromstring(xml_str)
    namespaces = {"w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}
    parrafos_comentados = []  # List to hold dictionaries

    for para in raiz.findall(".//w:p", namespaces):
        comments_start = para.findall(".//w:commentRangeStart", namespaces)
        comments_end = para.findall(".//w:commentRangeEnd", namespaces)

        if comments_start and comments_end:

            # Inicializa variables de almacenamiento
            texto_completo = ""
            texto_comentado = ""

            in_comment = False

            for child in para:
                if (
                    child.tag
                    == "{http://schemas.openxmlformats.org/wordprocessingml/2006/main}commentRangeStart"
                ):
                    in_comment = True

                if (
                    child.tag
                    == "{http://schemas.openxmlformats.org/wordprocessingml/2006/main}commentRangeEnd"
                ):
                    in_comment = False

                # Extrae elementos de un párrafo
                for elem in child.findall(".//w:t", namespaces):
                    texto_completo += elem.text

                    if in_comment:
                        texto_comentado += elem.text

            # Crea un diccionario
            para_dict = {
                "texto_completo": texto_completo,
                "texto_comentado": texto_comentado,
            }

            # Agrega el dicionario a la lista
            parrafos_comentados.append(para_dict)

    return parrafos_comentados


In [5]:
parrafos_comentados = extractor_parrafos(ruta_docx)
parrafos_comentados

[{'texto_completo': 'Prueba 02: este es un texto de prueba del texto de prueba 02',
  'texto_comentado': 'texto'},
 {'texto_completo': 'Prueba 04: este es un texto de prueba del texto de prueba 04',
  'texto_comentado': 'texto de prueba'}]

In [6]:
def extraer_comentarios(ruta_docx):
    """Función que extrae los comentarios de un docx
    Parámetro:
    ruta_docx: ruta del archivo

    Devuelve:
    lista: lista con los comentarios
    """
    # Inicializa el documento
    doc = Document(ruta_docx)

    # Enceuntra comentarios
    parte_comentarios = None
    for rel_id, rel in doc.part.rels.items():
        if "comments" in rel.target_ref:
            parte_comentarios = rel.target_part.blob
            break

    # Parsea la parte 'comments' del XML
    root = ET.fromstring(parte_comentarios)

    comentarios = []

    # Extraer texto de cada comentario en XML
    for comentario_xml in root.iter(
        "{http://schemas.openxmlformats.org/wordprocessingml/2006/main}comment"
    ):
        text = "".join(comentario_xml.itertext())
        comentarios.append(text)

    return comentarios


In [7]:
comentarios = extraer_comentarios(ruta_docx)
comentarios

['Comentario 01: este es un comentario al texto de Prueba 02',
 'Comentario 02: este es un comentario al texto de Prueba 04']

In [8]:
import pandas as pd

def crea_dataframe(ruta):
    """
    Crea un DataFrame a partir de las listas devueltas por extractor_parrafos y extraer_comentarios.

    Parámetro:
    ruta: ruta del archivo docx

    Devuelve:
    DataFrame de pandas con las columnas 'texto_completo', 'texto_comentado' y 'comentario'.
    """
    # Extrae párrafos comentados y comentarios del documento
    parrafos_comentados = extractor_parrafos(ruta)
    comentarios = extraer_comentarios(ruta)

    # Igualar longitudes de las listas
    len_diff = len(parrafos_comentados) - len(comentarios)

    if len_diff > 0:
        comentarios.extend([None] * len_diff)  # Agregar 'None' para igualar longitudes
    elif len_diff < 0:
        parrafos_comentados = parrafos_comentados[:len(comentarios)]  # Cortar lista para igualar longitudes

    # Juntar las listas en un DataFrame inicial
    zipped_lists = zip(parrafos_comentados, comentarios)
    df = pd.DataFrame(zipped_lists, columns=['Paragraphs', 'comentario'])

    # Dividir la columna 'Paragraphs' en 'texto_completo' y 'texto_comentado'
    df[['texto_completo', 'texto_comentado']] = df['Paragraphs'].apply(pd.Series)
    df.drop(columns=['Paragraphs'], inplace=True)  # Eliminar la columna 'Paragraphs' original

    return df


In [9]:
df = crea_dataframe(ruta_docx)
df

,comentario,texto_completo,texto_comentado
0,Comentario 01: este es un comentario al texto ...,Prueba 02: este es un texto de prueba del text...,texto
1,Comentario 02: este es un comentario al texto ...,Prueba 04: este es un texto de prueba del text...,texto de prueba
